<a target="_blank" href="https://colab.research.google.com/github/gox6/colab-demos/blob/main/rags/evaluate-rags-rigorously-or-perish.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [9]:
# Installing Python packages & hiding
!pip install --quiet langchain ragas xmltodict youtube-transcript-api pytube

In [5]:
# Importing the packages

import os

import chromadb
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context

In [6]:
# Managing secrets
# - If using Colab please use Colab Secrets
# - If running outside Colab please provide secrets as ENV VARS

import os

COLAB = os.getenv("COLAB_RELEASE_TAG") is not None

if COLAB:
  from google.colab import userdata
  OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
  os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
else:
  OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')



In [22]:
youtube_bbc_videos = ["https://youtu.be/YEHA-u8b43A","https://youtu.be/TlFfHjOMSXQ", "https://youtu.be/-kZolk9EoMA", "https://youtu.be/JQMZkrz6X08",
                      "https://youtu.be/D1iMZaLjBU4", "https://youtu.be/Sqcv9lCADxE",
                      "https://youtu.be/Cg3YMWcjLI4", "https://youtu.be/yQIMSv9Luw4", "https://youtu.be/5LYGnqoCLGk",
                      "https://youtu.be/lHggWT2iLdo", "https://youtu.be/TTUsxD62398", "https://youtu.be/3YMleRGjeqE", "https://youtu.be/h-5dqQMZTZQ",
                      "https://youtu.be/PtXzg6ykn8U", "https://youtu.be/QOUGlWEpwL4", "https://youtu.be/T05I-SBhXoI", "https://youtu.be/Iz-XY6XfXjk",
                      "https://youtu.be/yyLFQrb--pw", "https://youtu.be/wMqJbMPNM6A",
                      "https://youtu.be/FoBOSLofM3E", "https://youtu.be/97nEBjiQI1M", "https://youtu.be/bjmK4lGKNqY", "https://youtu.be/XWLA5A6bpwE",
                      "https://youtu.be/DkmrhVpCmac", "https://youtu.be/Dxar1d1aTUo"
                      ]


25

In [23]:
from collections import Counter
count = Counter(youtube_bbc_videos)
count

Counter({'https://youtu.be/YEHA-u8b43A': 1,
         'https://youtu.be/TlFfHjOMSXQ': 1,
         'https://youtu.be/-kZolk9EoMA': 1,
         'https://youtu.be/JQMZkrz6X08': 1,
         'https://youtu.be/D1iMZaLjBU4': 1,
         'https://youtu.be/Sqcv9lCADxE': 1,
         'https://youtu.be/Cg3YMWcjLI4': 1,
         'https://youtu.be/yQIMSv9Luw4': 1,
         'https://youtu.be/5LYGnqoCLGk': 1,
         'https://youtu.be/lHggWT2iLdo': 1,
         'https://youtu.be/TTUsxD62398': 1,
         'https://youtu.be/3YMleRGjeqE': 1,
         'https://youtu.be/h-5dqQMZTZQ': 1,
         'https://youtu.be/PtXzg6ykn8U': 1,
         'https://youtu.be/QOUGlWEpwL4': 1,
         'https://youtu.be/T05I-SBhXoI': 1,
         'https://youtu.be/Iz-XY6XfXjk': 1,
         'https://youtu.be/yyLFQrb--pw': 1,
         'https://youtu.be/wMqJbMPNM6A': 1,
         'https://youtu.be/FoBOSLofM3E': 1,
         'https://youtu.be/97nEBjiQI1M': 1,
         'https://youtu.be/bjmK4lGKNqY': 1,
         'https://youtu.be/XWLA5

In [11]:
from langchain_community.document_loaders import YoutubeLoader


loader = YoutubeLoader.from_youtube_url(
    "https://youtu.be/TlFfHjOMSXQ", add_video_info=True
)

docs = loader.load()

In [ ]:
client = chromadb.Client()


In [ ]:


# documents = load your documents

# generator with openai models
generator_llm = ChatOpenAI(model="gpt-3.5-turbo-16k")
critic_llm = ChatOpenAI(model="gpt-4")
embeddings = OpenAIEmbeddings()

generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    embeddings
)

# Change resulting question type distribution
distributions = {
    simple: 0,
    multi_context: 1,
    reasoning: 0
}

# use generator.generate_with_llamaindex_docs if you use llama-index as document loader
testset = generator.generate_with_langchain_docs(documents, 1, distributions)
testset.to_pandas()

embedding nodes:   0%|          | 0/20 [00:00<?, ?it/s]

Generating:   0%|          | 0/1 [00:00<?, ?it/s]

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What are the effects of PSO supplementation on...,"[['Muribaculaceae', 'Blautia']\nPunicic acid (...",PSO supplementation reduces hepatic lipid accu...,multi_context,"[{'uid': '38618675', 'Title': {'i': 'via', '#t...",True


In [ ]:
docs

[Document(page_content="smoking is the single biggest cause of preventable death in the UK it kills 2third of long-term users the aim of the new bill before Parliament is to protect younger Generations from harm by Banning tobacco use for them one caller with experience of that harm told the Nikki Campbell show on BBC Radio 5 live he regretted ever starting to smoke I have to have one lung removed got empia and COPD Neal lung and now I found out got terminal cancer in me Li and adrenal glands back I yeah it's some I've got used to now I've come to terms with it but what I've gone through the last three years I wouldn't like anybody to go through you know and ever could go back I wish I never started the harm done by smoking is well documented it causes lung and heart disease cancers still births dementia and Asthma England's chief medical officer sir Chris witty supports the ban people are trapped in smoking at a very young age uh and once they become addicted their choice is taken awa

In [ ]:
!pip install youtube-transcript-api pytube


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.2 MB/s eta 0:00:00
